In [1]:
import os
import warnings
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, "../models/")
from baseflow import baseflow_separation

from scipy.signal import find_peaks

from pandas.core.common import SettingWithCopyWarning

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
path_to_dataset = "/home/georgy/Documents/PEAK/results/tier2"

In [4]:
def snow_driven_or_not(states_instance):
    
    snowpack = states_instance["snowpack"].to_numpy()
    
    diffs = np.diff(np.flatnonzero(snowpack))
    
    periods = []
    
    length=0
    
    for i in range(1, len(diffs)):
        
        if diffs[i]==1:
            
            length = length + 1
            
            if i == len(diffs)-1:
                
                periods.append(length)
        
        else:
            periods.append(length)
            lenght = 0
    
    if np.any(np.array(periods)>30):
        snow_driven=True
    else:
        snow_driven=False
    
    return snow_driven

In [5]:
def flood_period_delineation(arr, threshold=0.1, duration=28):
    
    first=0
    last=0
    
    for i in range(len(arr)):
        
        if np.all(arr[i:i+duration]>threshold):
            
            first=i
            break
    
    for j in range(duration+1, 365-first):
        
        if np.all(arr[first:first+j]>threshold):
            
            last=first+j
    
    maxd = np.argmax(arr[first:last]) + first
    
    return first, last, maxd

def flood_period_delineation2(arr, threshold=0.5):
    
    positive_indexes = np.flatnonzero(arr>threshold)
    
    if len(positive_indexes) == 0:
        threshold = 0.1
        positive_indexes = np.flatnonzero(arr>threshold)
        
        if len(positive_indexes) == 0:
            threshold = 0.01
            positive_indexes = np.flatnonzero(arr>threshold)
            
            if len(positive_indexes) == 0:
                threshold = 0.
                positive_indexes = np.flatnonzero(arr>threshold)
        
    
    diffs = np.diff(positive_indexes)
    # marker of the end of the diffs to locate the last interval
    diffs = np.append(diffs, 999)
    
    lengths = []
    firsts = []
    lasts = []
    
    periods = []
    
    length=0
    first=positive_indexes[0]
    last=positive_indexes[0]
    
    for i in range(0, len(diffs)):
        
        if diffs[i]==1:
            
            length = length + 1
            last = last + 1
        
        else:
            lengths.append(length)
            firsts.append(first)
            lasts.append(last)
            periods.append([length, first, last])
            
            if i < len(diffs)-1: # not the last
                length = 0
                first=positive_indexes[i+1]
                last=positive_indexes[i+1]
            else: # the last value
                break
    
    max_dur_period = np.argmax(lengths)
    
    if firsts[max_dur_period] >=2:
        first = firsts[max_dur_period] - 2 # manual adjusting for higher threshold
    else:
        first = firsts[max_dur_period]
    
    last = lasts[max_dur_period]
    
    maxd = np.argmax(arr[first:last]) + first
    
    return first, last, maxd, periods


def baseflow_updater(baseflow_sep_instance, start, end, maxd):
    
    baseflow_sep_instance["Baseflow"] = baseflow_sep_instance["Baseflow_3"].copy()
    
    baseflow_sep_instance["Baseflow"].iloc[start:end] = np.nan
    
    baseflow_sep_instance["Baseflow"].iloc[maxd] = 0
    
    baseflow_sep_instance["Baseflow"] = baseflow_sep_instance["Baseflow"].interpolate()
    
    return baseflow_sep_instance.copy()

In [6]:
def flood_characteristics(baseflow_sep_instance, spring_flood=True):
    
    if spring_flood:
    
        quickflow = (baseflow_sep_instance["Streamflow"] - baseflow_sep_instance["Baseflow_3"]).to_numpy()

        # sf for spring flood
        sf_start, sf_end, sf_maxd, periods = flood_period_delineation2(quickflow) ###!!!! changed to "2" option

        # duration
        sf_dur = sf_end-sf_start

        # update Baseflow based on Russian theoretical studies (also in GrWAT)
        baseflow_sep_instance_upd = baseflow_updater(baseflow_sep_instance, sf_start, sf_end, sf_maxd)

        # sf runoff
        sf_Q = baseflow_sep_instance_upd["Streamflow"].iloc[sf_start:sf_end].copy()

        # total flood volume
        sf_vol = sf_Q.sum()

        # max runoff
        sf_maxQ = sf_Q.max()

        # prominent peaks
        sf_peaks, _ = find_peaks(sf_Q.to_numpy(), prominence=1)

        # number of prominent peaks
        sf_peaks_num = len(sf_peaks)

        # magnitude
        sf_mgn = sf_Q.max() - sf_Q.min()

        # ratio of sf volume in total flow
        sf_vol_ratio = sf_vol / baseflow_sep_instance_upd["Streamflow"].sum()
        
        # baseflow index (ratio of baseflow vol in total runoff)
        bfi = baseflow_sep_instance_upd["Baseflow"].sum() / baseflow_sep_instance_upd["Streamflow"].sum()
        
        sf_characteristics = [sf_start, sf_end, sf_dur, sf_vol, sf_vol_ratio,
                              sf_maxQ, sf_maxd, sf_peaks_num, sf_mgn, bfi]
        
        sf_characteristics_names = ["sf_start", "sf_end", "sf_dur", "sf_vol", "sf_vol_ratio",
                                    "sf_maxQ", "sf_maxd", "sf_peaks_num", "sf_mgn", "bfi"]
        
        ##########################################################################
        # rf for rain flood
        rf_dates = [False if (i > sf_start) and (i<sf_end) else True for i in range(len(quickflow))]
        
        # rain flood total volume 
        rf_vol = np.nansum(quickflow[rf_dates])
        
        # ration of rf in total flow
        rf_vol_ratio = rf_vol / baseflow_sep_instance_upd["Streamflow"].sum()
        
        # rf maxQ
        rf_maxQ = baseflow_sep_instance_upd["Streamflow"].iloc[rf_dates].max()
        
        # date of the max rain-induced flood
        rf_maxd = np.flatnonzero(baseflow_sep_instance_upd["Streamflow"] == rf_maxQ)[0]
        
        # peaks of rainfall-induced floods 
        rf_peaks, _ = find_peaks(baseflow_sep_instance_upd["Streamflow"].to_numpy(), prominence=0.1)
        
        # exclude peaks relevant to spring flood
        rf_peaks = [i for i in rf_peaks if i not in range(sf_start, sf_end+1)]
        
        # number of rf peaks
        rf_peaks_num = len(rf_peaks)
        
        # rainfall-induced flood periods (same threshold for spring flood)
        rf_periods = periods.copy()
        
        if len(rf_periods)>=2:
            rf_periods_lengths = [i[0] for i in rf_periods]

            # remove the spring flood period
            _ = rf_periods.pop(np.argmax(rf_periods_lengths))

            # updates period lengths
            rf_durations = np.array([i[0] for i in rf_periods])

            rf_periods_number = len(rf_durations)

            rf_duration_max = rf_durations.max()
            rf_duration_min = rf_durations.min()
            rf_duration_mean = rf_durations.mean()
            
        else:
            rf_periods_number = 0
            rf_duration_max = 0
            rf_duration_min = 0
            rf_duration_mean = 0
        
        rf_characteristics = [rf_vol, rf_vol_ratio, rf_maxQ, rf_maxd, rf_peaks_num, rf_periods_number,
                              rf_duration_max, rf_duration_min, rf_duration_mean]
        
        rf_characteristics_names = ["rf_vol", "rf_vol_ratio", "rf_maxQ", "rf_maxd", "rf_peaks_num", 
                                    "rf_periods_number", "rf_duration_max", "rf_duration_min", "rf_duration_mean"]
        
        #################################################################
        # misc
        
        # we have ration of baseflow in total, 
        # and we have a ration of rain-induced quickflow
        # so, we need spring-flood-induced ratio.
        # it will be different from sf_vol_ratio 
        # because sf_vol was calculated WITH baseflow
        
        sf_ratio = 1 - bfi - rf_vol_ratio
        
        # mean annual runoff
        mar = baseflow_sep_instance_upd["Streamflow"].mean()
        
        misc_characterstics = [sf_ratio, mar]
        
        misc_characterstics_names = ["sf_ratio", "mar"]
        
        characteristics = sf_characteristics + rf_characteristics + misc_characterstics
        
        characteristics_names = sf_characteristics_names + rf_characteristics_names + misc_characterstics_names
        
        return characteristics, characteristics_names
    
    else:
        
        quickflow = baseflow_sep_instance["Streamflow"] - baseflow_sep_instance["Baseflow_3"]
        
        # the ratio of
        flood_ratio = quickflow.sum() / baseflow_sep_instance["Streamflow"].sum()
        
        # bfi
        bfi = baseflow_sep_instance["Baseflow_3"].sum() / baseflow_sep_instance["Streamflow"].sum()
    
        # number of peaks
        peaks, _ = find_peaks(baseflow_sep_instance["Streamflow"].to_numpy(), prominence=0.1)
        peaks_num = len(peaks)
        
        # maximum flow
        maxQ = baseflow_sep_instance["Streamflow"].max()
        
        # the date of maximum flow
        maxd = np.flatnonzero(baseflow_sep_instance["Streamflow"] == maxQ)[0]
        
        # magnitude
        mgn = baseflow_sep_instance["Streamflow"].max() - baseflow_sep_instance["Streamflow"].min()
        
        # flood periods
        _,_,_,fl_periods = flood_period_delineation2(quickflow, threshold=0.5)
        
        # number of flood periods
        fl_periods_number = len(fl_periods)
        
        # duration of flood periods
        fl_durations = np.array([i[0] for i in fl_periods])
        
        # max, min, mean
        fl_duration_max = fl_durations.max()
        fl_duration_min = fl_durations.min()
        fl_duration_mean = fl_durations.mean()
        
        # mean annual runoff
        mar = baseflow_sep_instance["Streamflow"].mean()
               
        fl_characteristics = [flood_ratio, bfi, peaks_num, maxQ, maxd, mgn, 
                              fl_periods_number, fl_duration_max, fl_duration_min, 
                              fl_duration_mean, mar]
        
        fl_characteristics_names = ["flood_ratio", "bfi", "peaks_num", "maxQ", "maxd", "mgn",
                                    "fl_periods_number", "fl_duration_max", "fl_duration_min",
                                    "fl_duration_mean", "mar"]
        
        return fl_characteristics, fl_characteristics_names

In [7]:
def calculate_characteristics(basin_id, mode="HST", model="MIROC5", scenario="rcp26"):
    
    if mode == "HST":
        
        # define years to consider
        years = [str(i) for i in range(1979,2017)]
        
        path_runoff = os.path.join(path_to_dataset, "hydro/runoff/", mode, f"{basin_id}.csv")
        path_states = os.path.join(path_to_dataset, "hydro/states/", mode, f"{basin_id}.csv")
        
    elif mode == "PRJ":
        
        # define years to consider
        years = [str(i) for i in range(2016,2100)]
        
        path_runoff = os.path.join(path_to_dataset, "hydro/runoff/", mode, model, scenario, f"{basin_id}.csv")
        path_states = os.path.join(path_to_dataset, "hydro/states/", mode, model, scenario, f"{basin_id}.csv")
        
    # read input data
    runoff = pd.read_csv(path_runoff, 
                         index_col=0, 
                         parse_dates=True)

    states = pd.read_csv(path_states, 
                         index_col=0, 
                         parse_dates=True)

    # clip to historical period
    runoff = runoff[years[0]:years[-1]].copy()
    states = states[years[0]:years[-1]].copy()

    # hydrograph separation on quick- and baseflow
    baseflow_sep = baseflow_separation(runoff["Runoff"].copy())

    holder_snowdriven = {}
    holder_raindriven = {}

    cols_snowdriven = []
    cols_raindriven = []

    for year in years[1:]: # start from the first year + 1, e.g. 1980 *water* year
        
        #print(year)
        
        year = int(year)

        snow_driven = snow_driven_or_not(states[f"{year-1}-11-01":f"{year}-10-31"]) # 1 Nov -- 31 Oct

        characteristics, characteristics_names = flood_characteristics(baseflow_sep[f"{year-1}-11-01":f"{year}-10-31"], snow_driven)

        if snow_driven:

            holder_snowdriven[year] = characteristics
            cols_snowdriven = characteristics_names

        else:

            holder_raindriven[year] = characteristics
            cols_raindriven = characteristics_names

    output = []

    # convertion to pandas dataframes
    
    if len(holder_snowdriven)>0:
        df_snowdriven = pd.DataFrame(holder_snowdriven, index=cols_snowdriven).T
        output.append(df_snowdriven)
    else:
        output.append(pd.DataFrame())

    if len(holder_raindriven)>0:
        df_raindriven = pd.DataFrame(holder_raindriven, index=cols_raindriven).T
        output.append(df_raindriven)
    else:
        output.append(pd.DataFrame())
    
    return output

In [8]:
%%time
nvkz_sd, nvkz_rd = calculate_characteristics(10240, mode="HST")

CPU times: user 1.77 s, sys: 2.8 ms, total: 1.77 s
Wall time: 1.77 s


In [9]:
%%time
nvkz_sdf, nvkz_rdf = calculate_characteristics(10240, mode="PRJ")

CPU times: user 3.83 s, sys: 3.56 ms, total: 3.83 s
Wall time: 3.83 s
